## Heston Model:


\begin{cases} 
dS_t = \mu S_t\;dt\:+\:\sqrt{\nu_t}S_t\;dW_1 \\
d\nu_t = \theta(\omega\:-\nu_t)dt + \xi\sqrt{\nu_t}\:dW_2 
\\
(dW_1, dW_2) = \rho\:dt
\end{cases}







In [ ]:
import numpy as np
i = complex(0, 1)

# To be used in the Heston pricer
def fHeston(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    # To be used a lot
    prod = rho * sigma * i * s
    # Calculate d
    d1 = (prod - kappa)**2
    d2 = (sigma**2) * (i * s + s**2)
    d = np.sqrt(d1 + d2)
    # Calculate g
    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1 / g2
    # Calculate first exponential
    exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
    exp2 = 1 - g * np.exp(-d * T)
    exp3 = 1 - g
    mainExp1 = exp1 * np.power(exp2 / exp3, -2 * theta * kappa/(sigma**2))
    # Calculate second exponential
    exp4 = theta * kappa * T / (sigma**2)
    exp5 = volvol / (sigma**2)
    exp6 = (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    mainExp2 = np.exp((exp4 * g1) + (exp5 * g1 * exp6))
    return (mainExp1 * mainExp2)

# Heston Pricer
def priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho):
    P, iterations, maxNumber = 0, 1000, 100
    ds = maxNumber / iterations
    element1 = 0.5 * (St - K * np.exp(-r * T))
    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i
        numerator1 = fHeston(s2, St, K, r, T, 
                             sigma, kappa, theta, volvol, rho)
        numerator2 = K * fHeston(s1, St, K, r, T, 
                               sigma, kappa, theta, volvol, rho)
        denominator = np.exp(np.log(K) * i * s1) * i * s1
        P += ds * (numerator1 - numerator2) / denominator
    element2 = P / np.pi
    return np.real((element1 + element2))

In [ ]:
# current price
St = 100
# strike
K = 80
# risk free interest rate
r = 0.1
# time to maturity
T = 1
# volatility 
sigma = 0.8
# kappa = ?
kappa = 0.2
# theta = ?
theta = 0.1
# volatility of volatility
volvol = 0.3
# rho = ?
rho = 0.4

In [ ]:
priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho)

31.98973380102906